In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elrii\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
input_dir = r'./input/'
output_dir = r'./output/'

In [41]:
dfa = pd.read_excel(input_dir+"2017-COVID_audiograms.xlsx")
dfa = dfa[['SEQN','AUXU1K1R', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU1K1L','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']].replace(666,100)

audiogram_col = ['AUXU1K1R', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU1K1L','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']
audiogram_right = audiogram_col[:8]
audiogram_left = audiogram_col[8:]

dfa = dfa[~dfa[audiogram_col].isin([888]).any(axis=1)].dropna().astype(int)
dfa = dfa[(dfa[audiogram_right].sum(axis=1) >= 160) & (dfa[audiogram_right].sum(axis=1) >= 160)].reset_index(drop=True)
dfa

,SEQN,AUXU1K1R,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,AUXU8KR,AUXU1K1L,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL
0,109282,10,10,10,10,30,30,30,70,10,0,10,5,15,25,40,60
1,109330,10,20,10,10,35,40,55,50,10,20,10,30,35,35,55,60
2,109361,30,30,30,35,25,20,15,5,10,10,10,15,15,10,10,10
3,109367,30,30,30,70,75,80,90,85,55,35,55,65,70,75,75,100
4,109382,30,45,35,25,15,25,45,60,25,40,20,20,15,20,25,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,124756,5,5,5,10,30,35,30,60,15,10,10,10,25,25,25,40
1232,124784,45,45,45,55,45,50,40,60,50,45,50,50,50,50,45,80
1233,124799,30,25,35,40,35,55,70,100,40,20,40,45,45,60,85,100
1234,124811,20,30,20,5,25,15,30,45,30,35,35,15,30,25,55,60


In [42]:
dfq = pd.read_excel(input_dir+"2017-COVID_questionaire.xlsx")
dfq = dfq.dropna(thresh=5)
dfq=dfq.merge(dfa['SEQN'], on='SEQN', how='inner')
dfq.shape

(1236, 58)

In [43]:
dfa=dfa.merge(dfq['SEQN'], on='SEQN', how='inner')
dfa.shape

(1236, 17)

In [51]:
dfd = pd.read_excel(input_dir+"2017-COVID_demographics.xlsx")
dfd = dfd[['SEQN', 'RIAGENDR','RIDAGEYR']] #1 = male, 2 = female
dfd=dfd.merge(dfq['SEQN'], on='SEQN', how='inner').astype(int)

thresholds = [0, 25, 35, 45, 55, dfd['RIDAGEYR'].max()] 
labels = [1, 2, 3,4,5]
dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'], 
                         bins=thresholds, labels=labels, right=True)
dfd.shape

(1236, 3)

In [52]:
dfV = dfd.merge(dfa, on='SEQN', how='inner')#.merge(dfq, on='SEQN', how='inner')
dfV.shape
dfV.to_excel(output_dir+"N")

(1236, 19)

In [53]:
dfV

,SEQN,RIAGENDR,RIDAGEYR,AUXU1K1R,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,AUXU8KR,AUXU1K1L,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL
0,109282,1,5,10,10,10,10,30,30,30,70,10,0,10,5,15,25,40,60
1,109330,1,5,10,20,10,10,35,40,55,50,10,20,10,30,35,35,55,60
2,109361,1,1,30,30,30,35,25,20,15,5,10,10,10,15,15,10,10,10
3,109367,1,5,30,30,30,70,75,80,90,85,55,35,55,65,70,75,75,100
4,109382,2,5,30,45,35,25,15,25,45,60,25,40,20,20,15,20,25,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,124756,2,5,5,5,5,10,30,35,30,60,15,10,10,10,25,25,25,40
1232,124784,2,5,45,45,45,55,45,50,40,60,50,45,50,50,50,50,45,80
1233,124799,2,5,30,25,35,40,35,55,70,100,40,20,40,45,45,60,85,100
1234,124811,1,5,20,30,20,5,25,15,30,45,30,35,35,15,30,25,55,60
